In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import torch

from transformers.pipelines import pipeline
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, CrossEncoder, LoggingHandler, losses, InputExample, util

In [126]:
cv = pd.read_csv('data/hh_cv.csv', sep='\t')

col_names = cv.columns.values[0].split(';')
cv.columns = ['info']
cv[col_names] = cv['info'].str.split(';', expand=True).iloc[:,:len(col_names)]

In [127]:
cv.head(3)

,info,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979;290...","Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...","""Опыт работы 16 лет 10 месяцев Август 2010 — ...","""МАОУ """"СОШ № 1 г.Немана""""""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959;4000...","Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982;2...","Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано


In [128]:
cv.nunique()

info                               44591
Пол, возраст                       16003
ЗП                                   690
Ищет работу на должность:          14912
Город, переезд, командировки       10147
Занятость                            141
График                               101
Опыт работы                        44333
Последнее/нынешнее место работы    35025
Последняя/нынешняя должность       25948
Образование и ВУЗ                  40621
Обновление резюме                  29503
Авто                               15248
dtype: int64

In [129]:
cv = cv[['Ищет работу на должность:', 'ЗП', 'Опыт работы', 'Занятость', 'График', 'Город, переезд, командировки']]

In [130]:
cv.head(3)

,Ищет работу на должность:,ЗП,Опыт работы,Занятость,График,"Город, переезд, командировки"
0,Системный администратор,29000 руб.,"""Опыт работы 16 лет 10 месяцев Август 2010 — ...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...","Советск (Калининградская область) , не готов к..."
1,Технический писатель,40000 руб.,Опыт работы 19 лет 5 месяцев Январь 2000 — по...,"частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...","Королев , не готов к переезду , готов к редким..."
2,Оператор,20000 руб.,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,полная занятость,полный день,"Тверь , не готова к переезду , не готова к ком..."


In [131]:
vacancy = pd.read_csv('data/hh_vacancy.csv', on_bad_lines='skip', encoding='utf-8', sep=';')

In [132]:
vacancy = vacancy.dropna()

In [133]:
vacancy.head(3)

,№,id,title,salary,experience,job_type,description,key_skills,company,location,date_of_post,type
6,7,78652831,Начальник участка,от 130000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",В ГК МИЛАНСТРОЙ требуется начальник участка!...,"AutoCAD,MS Excel,MS Word,Управление производст...",Артюхова Анастасия Александровна,"Москва, улица Ивана Бабушкина, 4",29.03.2023,close
7,8,69339467,Ведущий архитектор,от 80000 до 100000 RUR,От 1 года до 3 лет,"Полная занятость,полный день",ГК «ОЛИМПРОЕКТ» - лидирующая компания в сф...,"AutoCAD,Autodesk Revit,Работа в команде,Adobe ...",ГК Олимпроект,"Владивосток, Океанский проспект, 17",13.04.2023,close
8,9,78146411,Директор по персоналу (HRD),от 250000 до 350000 RUR,Более 6 лет,"Полная занятость,удаленная работа",Компания «Ортоника» является одним из веду...,"OKR,KPI,Грейдинг,Онбординг,компетентностный по...",Ортоника,"Владимир, Октябрьский проспект, 22",22.04.2023,close


In [134]:
vacancy.drop(['№', 'id', 'company', 'date_of_post', 'type', 'description'], axis=1, inplace=True)

In [135]:
vacancy.head(3)

,title,salary,experience,job_type,key_skills,location
6,Начальник участка,от 130000 RUR,От 1 года до 3 лет,"Полная занятость,полный день","AutoCAD,MS Excel,MS Word,Управление производст...","Москва, улица Ивана Бабушкина, 4"
7,Ведущий архитектор,от 80000 до 100000 RUR,От 1 года до 3 лет,"Полная занятость,полный день","AutoCAD,Autodesk Revit,Работа в команде,Adobe ...","Владивосток, Океанский проспект, 17"
8,Директор по персоналу (HRD),от 250000 до 350000 RUR,Более 6 лет,"Полная занятость,удаленная работа","OKR,KPI,Грейдинг,Онбординг,компетентностный по...","Владимир, Октябрьский проспект, 22"


### Semantic search

#### multilingual-e5-large

In [61]:
import dill

In [33]:
model_name = 'intfloat/multilingual-e5-large'

retrieval_model = SentenceTransformer(model_name)

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [49]:
df_vacancies_concat = vacancy.apply(lambda row: '. '.join(row.astype(str)), axis=1)

In [52]:
df_vacancies_concat.values[0]

'Начальник участка. от 130000 RUR. От 1 года до 3 лет. Полная занятость,полный день. AutoCAD,MS Excel,MS Word,Управление производственным персоналом,Составление графиков. Москва, улица Ивана Бабушкина, 4'

In [93]:
vacancies_list = df_vacancies_concat.values.tolist()

In [59]:
%%time

vacancies_embedding_e5 = retrieval_model.encode(vacancies_list, convert_to_tensor=True, device='cuda')

In [153]:
with open('emb/vacancies_embedding_e5_v1.dill', 'wb') as f:
    dill.dump(vacancies_embedding_e5, f)

In [150]:
def sem_search_results(model, query, corpus, corpus_embedding, top_k):
    #Sematic Search (Retrieve)
    question_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embedding, top_k=top_k)
    hits = hits[0] 

    cross_inp_res = [[query, corpus[hit['corpus_id']], hit['score']] for hit in hits]
    cross_inp_emb = [question_embedding[0]] + [corpus_embedding[hit['corpus_id']] for hit in hits]
    return cross_inp_res, cross_inp_emb

In [152]:
for i in range(3):
    query = cv.iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    cross_inp_res, cross_inp_emb = sem_search_results(retrieval_model, query, vacancies_list, vacancies_embedding_e5, 5)
    print(query)
    print()
    print(cross_inp_res[0][1])
    print('--------------------------------------------------------------------------------------------------------------------')

Системный администратор. 29000 руб.. "Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ ""СОШ № 1 г.Немана"" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС ""ВЕСТЕР-ИНФО"" Старший продавец, директор отдела Продажи компьютерной техники". частичная занятость, проектная работа, полная занятость. гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. Советск (Калининградская область) , не готов к переезду , не готов к командировкам

Системный администратор. от 90000 до 140000 RUR. Более 6 лет. Полная занятость,полный день. Администрирование сетевого оборудования,Настройка сетевых подключений,Информационные технологии,Linux,Networking,Маршрутизация статическая,Маршрутизация динамическая,Телекоммуникации,Сетевые технологии,Высокий уровень самоорганизации,Умею начинать дела и довод

### Обработка

#### Резюме

In [2]:
import re
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm

In [3]:
nlp = spacy.load("ru_core_news_sm")

C:\Users\User\anaconda3\envs\myenv_py_3_10\lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
cv = pd.read_csv('data/hh_cv.csv', on_bad_lines='skip', encoding='utf-8', sep=';')

In [5]:
cv['job_type'] = cv['Занятость'] + ', ' + cv['График']

In [6]:
cv = cv.drop(['Пол, возраст', 'Обновление резюме', 'Авто', 'Последнее/нынешнее место работы', 'Последняя/нынешняя должность', 'Образование и ВУЗ', 'Занятость', 'График'], axis=1)

In [7]:
cv.head(3)

,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Опыт работы,job_type
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"частичная занятость, проектная работа, полная ..."
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,"частичная занятость, проектная работа, полная ..."
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,"полная занятость, полный день"


In [495]:
cv.shape

(44744, 8)

In [387]:
cv.isna().sum()

ЗП                                0
Ищет работу на должность:         0
Город, переезд, командировки      0
Опыт работы                     168
job_type                          0
dtype: int64

In [388]:
cv = cv.rename(columns={
    'Ищет работу на должность:':'title',
    'ЗП':'salary',
    'Город, переезд, командировки':'location',
    'График':'job_type',
    'Опыт работы':'experience',
    'Последнее/нынешнее место работы':'last_job'
})

In [389]:
cv.salary = cv.salary.apply(lambda value: value.split(' ')[0])

In [390]:
cv.location = cv.location.apply(lambda value: value.split(' ')[0])

In [391]:
def find_experience(experience):
    if experience=='Не указано':
        return -1
    else:
        result_exp = re.findall(pattern, experience)[0]
        mnth = int(result_exp[0])*12 + int(result_exp[1])
        return mnth
    
def experience_to_group(months):
    if months < 12:
        return 'Нет опыта'
    elif 12 <= months < 36:
        return 'От 1 года до 3 лет'
    elif 36 <= months < 72:
        return 'От 3 до 6 лет'
    else:
        return 'Более 6 лет'

cv.experience = cv.experience.fillna('0 0')
cv['experience_month'] = cv.apply(lambda row: find_experience(row.experience), axis=1)
cv['experience_group'] = cv.experience_month.apply(experience_to_group)

In [492]:
stop_words = set(stopwords.words('russian') + ['опыт', 'работы', 'месяца', 'месяцев', 'год', 'года', 'лет','настоящее','время'])

def extract_skills(text):
    doc = nlp(text)
    skills = [token.text for token in doc if token.pos_ in {'NOUN', 'ADJ'} and token.text.lower() not in stop_words]
    return ' '.join(skills)

cv['experience_clean'] = [extract_skills(i) for i in tqdm(cv.experience.values)]


  1%|▏         | 562/44744 [01:31<2:31:49,  4.85it/s]


  2%|▏         | 1098/44744 [03:04<4:17:58,  2.82it/s]


  4%|▎         | 1615/44744 [04:28<3:47:57,  3.15it/s]


  5%|▍         | 2120/44744 [06:00<1:24:18,  8.43it/s]


  6%|▌         | 2653/44744 [07:24<1:09:23, 10.11it/s]


...

 97%|█████████▋| 43526/44744 [3:07:50<05:28,  3.71it/s]


 98%|█████████▊| 43914/44744 [3:09:58<03:29,  3.97it/s]


 99%|█████████▉| 44266/44744 [3:12:18<02:24,  3.32it/s]


100%|█████████▉| 44617/44744 [3:14:29<00:39,  3.23it/s]


100%|██████████| 44744/44744 [3:15:13<00:00,  3.82it/s]


In [493]:
cv.to_csv('cv_clean.csv')

#### Вакансии

In [8]:
from nltk.corpus import stopwords
from tqdm import tqdm
import pandas as pd
import spacy
import re

In [9]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
vacancy = pd.read_csv('data/hh_vacancy.csv', on_bad_lines='skip', encoding='utf-8', sep=';')
vacancy = vacancy.dropna()
vacancy = vacancy.drop(['company','date_of_post','type','№','id'], axis=1)

vacancy = vacancy.rename(columns={
    'experience':'experience_group'
})

def extract_average_salary(salary_range):
    # Поиск числовых значений в строке
    numbers = [int(match.group()) for match in re.finditer(r'\b\d+\b', salary_range)]
    
    # Если числа найдены, возвращаем среднее значение, иначе возвращаем None
    if numbers:
        return sum(numbers) / len(numbers)
    else:
        return None
    
vacancy.salary = vacancy.salary.apply(extract_average_salary)

vacancy.location = vacancy.location.apply(lambda value: value.replace('г.', '').split(',')[0])

In [ ]:
stop_words = set(stopwords.words('russian') + ['опыт', 'работы', 'месяца', 'месяцев', 'год', 'года', 'лет','настоящее','время'])

def extract_skills(text):
    doc = nlp(text)
    skills = [token.text for token in doc if token.pos_ in {'NOUN', 'ADJ'} and token.text.lower() not in stop_words]
    return ' '.join(skills)

vacancy['description_clean'] = [extract_skills(i) for i in tqdm(vacancy.description.values)]

In [ ]:
vacancy.to_csv('vacancy_clean.csv')

### Semantic search

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import torch
import re 
import dill
import spacy

from transformers.pipelines import pipeline
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, CrossEncoder, LoggingHandler, losses, InputExample, util

In [2]:
nlp = spacy.load("ru_core_news_sm")

C:\Users\User\anaconda3\envs\myenv_py_3_10\lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [3]:
def extract_average_salary(salary_range):
    # Поиск числовых значений в строке
    numbers = [int(match.group()) for match in re.finditer(r'\b\d+\b', salary_range)]
    
    # Если числа найдены, возвращаем среднее значение, иначе возвращаем None
    if numbers:
        return sum(numbers) / len(numbers)
    else:
        return None

In [4]:
model_name = 'intfloat/multilingual-e5-large'

model = SentenceTransformer(model_name)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [6]:
cv = pd.read_csv('cv_clean.csv', index_col=0)

cv = cv.rename(columns={
    'experience_clean_list':'description'
})

cv.salary = cv.salary.astype('float64')

cv.job_type = cv.job_type.apply(lambda value: value.capitalize())

cv.description = cv.description.fillna('')
cv.description = cv.description.apply(lambda value: ''.join([i for i in value if not i.isdigit()]))
cv.description = cv.description.apply(lambda value: ' '.join(value.split()))

In [7]:
vacancy = pd.read_csv('data/hh_vacancy.csv', on_bad_lines='skip', encoding='utf-8', sep=';')

vacancy = vacancy.dropna()
vacancy = vacancy.drop(['company','date_of_post','type','№','id'], axis=1)

vacancy.salary = vacancy.salary.apply(extract_average_salary)
vacancy.location = vacancy.location.apply(lambda value: value.replace('г.', '').split(',')[0])
vacancy.job_type = vacancy.job_type.apply(lambda value: ', '.join(value.split(',')))

vacancy = vacancy.rename(columns={
    'experience':'experience_group'
})
vacancy['description'] = vacancy['description'] + ' ' + vacancy['key_skills']

In [8]:
cols = ['title', 'location', 'salary', 'experience_group', 'job_type', 'description']

cv = cv[cols]
vacancy = vacancy[cols]

In [9]:
print('cv: ', cv.shape)
print('vacancy: ', vacancy.shape)

cv:  (44744, 6)
vacancy:  (185183, 6)


In [ ]:
def exp_group(value):
    if value=='Нет опыта':
        return 1
    elif value=='От 1 года до 3 лет':
        return 2
    elif value=='От 3 до 6 лет':
        return 3
    else:
        return 4

cv['exp_group_num'] = cv.experience_group.apply(exp_group)
vacancy['exp_group_num'] = vacancy.experience_group.apply(exp_group)

In [10]:
cv.head(3)

,title,location,salary,experience_group,job_type,description
0,Системный администратор,Советск,29000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",СОШ г Немана Системный администратор установка...
1,Технический писатель,Королев,40000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",Временный трудовой коллектив Информационные те...
2,Оператор,Тверь,20000.0,Более 6 лет,"Полная занятость, полный день",операционист денежной наличностью обслуживание...


In [11]:
cv.nunique()

title               14929
location              958
salary                498
experience_group        4
job_type              468
description         43851
dtype: int64

In [12]:
vacancy.head(3)

,title,location,salary,experience_group,job_type,description
6,Начальник участка,Москва,130000.0,От 1 года до 3 лет,"Полная занятость, полный день",В ГК МИЛАНСТРОЙ требуется начальник участка!...
7,Ведущий архитектор,Владивосток,90000.0,От 1 года до 3 лет,"Полная занятость, полный день",ГК «ОЛИМПРОЕКТ» - лидирующая компания в сф...
8,Директор по персоналу (HRD),Владимир,300000.0,Более 6 лет,"Полная занятость, удаленная работа",Компания «Ортоника» является одним из веду...


In [13]:
vacancy.nunique()

title                79840
location              4848
salary                3922
experience_group         4
job_type                21
description         172519
dtype: int64

In [14]:
cv.description.values[0]

'СОШ г Немана Системный администратор установка ремонт периферийной техники локальная сеть Ведение Электронного журнала сайта организации Август месяц ВЕСТЕР - ИНФО Старший продавец директор отдела компьютерной техники'

In [15]:
vacancy.description.values[0]

'  В ГК МИЛАНСТРОЙ требуется начальник участка!    _  Факты о нас:  _    1. _Более 15 лет успешной работы в сфере строительства _   2. _Стабильная надежная компания: мы выполняем все свои обязательства как перед клиентами, так и перед сотрудниками._   3. _Мы охватываем все виды работ от проектирования до сдачи объекта в эксплуатацию _   4. _Наша команда - более 700 человек _   5. _Более 250 видов техники в распоряжении компании _   6. _Мы беремся за социально значимые проекты: детские сады/школы/спортивные комплексы _   7. _Наши заказчики - департаменты Правительства г. Москвы, а также другие гос. структуры._    Обязанности:    —Организация строительного производства на площадке   — Расстановка бригад   — Контроль качества производства работ   — Контроль за ОТ и ТБ   — Работа с субподрядными организациями и бригадами   — Составление графиков производства работ   — Составление месячных заявок на материалы и контроль их исполнения   — Взаимодействие с заказчиком и авторским надзором по в

#### Объединяем все столбцы

In [21]:
def sem_search_results(model, query, corpus, corpus_embedding, top_k):
    #Sematic Search (Retrieve)
    question_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embedding, top_k=top_k)
    hits = hits[0] 
    
    print(question_embedding.shape)
    print(corpus_embedding.shape)
    
    cross_inp_res = [[query, corpus[hit['corpus_id']], hit['score']] for hit in hits]
    cross_inp_emb = [question_embedding[0]] + [corpus_embedding[hit['corpus_id']] for hit in hits]
    return cross_inp_res, cross_inp_emb

def extract_skills(text):
    doc = nlp(text)
    skills = [token.text for token in doc if token.pos_ in {'NOUN', 'ADJ'} and token.text.lower() not in stop_words]
    return ' '.join(skills)

stop_words = set(stopwords.words('russian') + ['опыт', 'работы', 'месяца', 'месяцев', 'год', 'года', 'лет','настоящее','время'])

In [17]:
%%time

df_cv_concat = cv.iloc[:1000].apply(lambda row: '. '.join(row.astype(str)), axis=1)

cv_list = df_cv_concat.values.tolist()

cv_embedding_e5 = model.encode(cv_list, convert_to_tensor=True, device='cuda')

CPU times: total: 17.5 s
Wall time: 15.8 s


In [18]:
with open('emb/cv_embedding_e5.dill', 'wb') as f:
    dill.dump(cv_embedding_e5, f)

In [31]:
for i in range(3):
    query = vacancy.iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    
    clean_query = ''.join([i for i in query if not i.isdigit()])
    clean_query = extract_skills(clean_query)
    
    cross_inp_res, cross_inp_emb = sem_search_results(model, clean_query, cv_list, cv_embedding_e5, 5)
    print(query)
    print()
    print(cross_inp_res[0][1])
    print('--------------------------------------------------------------------------------------------------------------------')

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день.   В ГК МИЛАНСТРОЙ требуется начальник участка!    _  Факты о нас:  _    1. _Более 15 лет успешной работы в сфере строительства _   2. _Стабильная надежная компания: мы выполняем все свои обязательства как перед клиентами, так и перед сотрудниками._   3. _Мы охватываем все виды работ от проектирования до сдачи объекта в эксплуатацию _   4. _Наша команда - более 700 человек _   5. _Более 250 видов техники в распоряжении компании _   6. _Мы беремся за социально значимые проекты: детские сады/школы/спортивные комплексы _   7. _Наши заказчики - департаменты Правительства г. Москвы, а также другие гос. структуры._    Обязанности:    —Организация строительного производства на площадке   — Расстановка бригад   — Контроль качества производства работ   — Контроль за ОТ и ТБ   — Работа с субподрядными организациями и бригадами   — Составление графиков производства работ   — Составление месячных заявок на матер

Директор по персоналу (HRD). Владимир. 300000.0. Более 6 лет. Полная занятость, удаленная работа. Компания   «Ортоника»   является одним из ведущих российских производителей современных средств реабилитации.    Компания представлена в различных городах России с Центральным офисом в городе Владимир.    Ортоника активно развивается, внедряя новые технологии. А поэтому у нас все больше задач, связанных с управлением персонала.    В связи этим приглашаем современного Директора по персоналу, но прежде чем рассказать с какой целью, давайте на минуту погрузимся в нашу историю...      Почему мы:      «Ортоника» интенсивно растет даже в наши непростые времена,   предоставляя превосходную возможность раскрыть и применить свои таланты с оплатой труда выше рыночной. Для этого у нас уже есть сплоченная команда HR, которую мы планируем пополнять и развивать. А также есть вызовы, для которых и нужен новый лидер HR - к нам приходят целыми командами сотрудники, которых необходимо адаптировать, мотивиро

In [19]:
%%time

df_cv_concat = cv.iloc[:1000].drop(['description'], axis=1).apply(lambda row: '. '.join(row.astype(str)), axis=1)

cv_list = df_cv_concat.values.tolist()

cv_embedding_e5 = model.encode(cv_list, convert_to_tensor=True, device='cuda')

CPU times: total: 2.08 s
Wall time: 1.81 s


##### Уберем описание

In [22]:
for i in range(3):
    query = vacancy.drop(['description'], axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    
    clean_query = ''.join([i for i in query if not i.isdigit()])
    clean_query = extract_skills(clean_query)
    
    cross_inp_res, cross_inp_emb = sem_search_results(model, clean_query, cv_list, cv_embedding_e5, 5)
    print(query)
    print()
    print(cross_inp_res[0][1])
    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([1024])
torch.Size([1000, 1024])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день

Начальник отдела. Красноярск. 40000.0. От 3 до 6 лет. Полная занятость, полный день
--------------------------------------------------------------------------------------------------------------------
torch.Size([1024])
torch.Size([1000, 1024])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день

Архитектор ПО / Ведущий Разработчик. Санкт-Петербург. 250000.0. Более 6 лет. Полная занятость, гибкий график, полный день, удаленная работа
--------------------------------------------------------------------------------------------------------------------
torch.Size([1024])
torch.Size([1000, 1024])
Директор по персоналу (HRD). Владимир. 300000.0. Более 6 лет. Полная занятость, удаленная работа

Руководитель отдела проектов. Ростов-на-Дону. 40000.0. Более 6 лет. Полная занятость, полный день, удаленная работа
---------------

### FastText

In [109]:
import fasttext
import tensorflow as tf

In [110]:
ft = fasttext.load_model('C:/Users/User/Documents/Projects/itmo_rabota_ru/cc.ru.300.bin')

In [111]:
cv.head(3)

,title,location,salary,experience_group,job_type,description,exp_group_num
0,Системный администратор,Советск,29000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",СОШ г Немана Системный администратор установка...,4
1,Технический писатель,Королев,40000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",Временный трудовой коллектив Информационные те...,4
2,Оператор,Тверь,20000.0,Более 6 лет,"Полная занятость, полный день",операционист денежной наличностью обслуживание...,4


In [112]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

def list_to_tensor(list_of_lists):
    list_of_lists = np.array([np.array(xi) for xi in list_of_lists])
    device = get_device()
    return torch.from_numpy(list_of_lists).float().to(device)

def concat_tensors(tensors):
    return torch.cat(tensors, 1)

def get_emb_fasttext(data, cols):
    col_list = data[cols].apply(lambda row: '. '.join(row.astype(str)), axis=1).to_list()
    
    vectors = []

    for words in col_list:
        vector = np.mean([ft.get_word_vector(word) for word in words.split(' ')], axis=0).tolist()
        vectors.append(vector)
    return list_to_tensor(vectors)

In [155]:
get_emb_fasttext(cv.iloc[:1000], cols=['title', 'location', 'job_type']).shape

torch.Size([1000, 300])

In [113]:
cv_numb_ft = df_to_tensor(cv[['salary','exp_group_num']])
cv_other_ft = get_emb_fasttext(cv, cols=['title', 'location', 'job_type'])
cv_other_all_ft = get_emb_fasttext(cv, cols=['salary','exp_group_num','title', 'location', 'job_type'])
cv_desc_ft = get_emb_fasttext(cv, cols=['description'])

In [114]:
vacancy = vacancy.dropna()

### FastText. Числовые столбцы + Текстовые эмб отдельно

In [119]:
cv_tensor = concat_tensors((cv_numb_ft, cv_other_ft))

for i in range(3):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_emb_fasttext(query, cols=['title', 'location', 'job_type'])
    query_other_all = get_emb_fasttext(query, cols=['salary','exp_group_num','title', 'location', 'job_type'])
    query_desc = get_emb_fasttext(query, cols=['description'])
    
    query_tensor = concat_tensors((query_numb, query_other))[0]
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_tensor, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(3):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([302])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. 4', 1.0]

['Технический писатель. Королев. 40000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, удаленная работа. 4', 1.0]

['Оператор. Тверь. 20000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

--------------------------------------------------------------------------------------------------------------------
torch.Size([302])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа

### FastText. Числовые столбцы + Текстовые эмб отдельно + Описание

In [120]:
cv_tensor = concat_tensors((cv_numb_ft, cv_other_ft, cv_desc_ft))

for i in range(3):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_emb_fasttext(query, cols=['title', 'location', 'job_type'])
    query_other_all = get_emb_fasttext(query, cols=['salary','exp_group_num','title', 'location', 'job_type'])
    query_desc = get_emb_fasttext(query, cols=['description'])
    
    query_tensor = concat_tensors((query_numb, query_other, query_desc))[0]
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_tensor, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(3):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([602])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. 4', 1.0]

['Технический писатель. Королев. 40000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, удаленная работа. 4', 1.0]

['Оператор. Тверь. 20000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

--------------------------------------------------------------------------------------------------------------------
torch.Size([602])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа

### FastText. Описание

In [121]:
cv_tensor = cv_other_ft

for i in range(3):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_emb_fasttext(query, cols=['title', 'location', 'job_type'])
    query_other_all = get_emb_fasttext(query, cols=['salary','exp_group_num','title', 'location', 'job_type'])
    query_desc = get_emb_fasttext(query, cols=['description'])
    
    query_tensor = query_other[0]
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_tensor, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(3):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([300])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Начальник участка. Москва. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9853485822677612]

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9853485822677612]

--------------------------------------------------------------------------------------------------------------------
torch.Size([300])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий программист 1C. Владивосток. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9849994778633118]

['Ведущий программист 1C. Владивосток. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9849994778633118]

['Ведущий специалист-руководитель. Санкт-Петербург. 80000.0. Более 6 лет

In [171]:
cv_tensor = cv_other_all_ft

print('FastText | БЕЗ ОПИСАНИЯ')
print()
for i in range(4):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_emb_fasttext(query, cols=['title', 'location', 'job_type'])
    query_other_all = get_emb_fasttext(query, cols=['salary','exp_group_num','title', 'location', 'job_type'])
    query_desc = get_emb_fasttext(query, cols=['description'])
    
    query_tensor = query_other_all[0]
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_tensor, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

FastText | БЕЗ ОПИСАНИЯ

torch.Size([300])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Руководитель отдела. Москва. 80000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9865855574607849]

['Руководитель проектов. Москва. 50000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9831064343452454]

--------------------------------------------------------------------------------------------------------------------
torch.Size([300])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий программист. Калининград. 100000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9918026924133301]

['Ведущий специалист. Ставрополь. 30000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9916707277297974]

--------------------------------------------------------------------------------------------------------------------
torch.Size([300])
Директор по персоналу (HR

### multilingual-e5-large

In [28]:
def get_e5_emb(data):
    data_list = data.apply(lambda row: '. '.join(row.astype(str)), axis=1).tolist()
    enc = model.encode(data_list, convert_to_tensor=True, device='cuda')
    return enc

In [65]:
cv_list = cv.drop('description',axis=1).apply(lambda row: '. '.join(row.astype(str)), axis=1).values

In [29]:
%%time

cv_numb = df_to_tensor(cv[['salary','exp_group_num']])
cv_other = get_e5_emb(cv[['title', 'location', 'job_type']])
cv_other_all = get_e5_emb(cv[['salary','exp_group_num','title', 'location', 'job_type']])
cv_desc = get_e5_emb(cv[['description']])

CPU times: total: 15min 37s
Wall time: 13min 10s


#### multilingual-e5-large. Числовые + Текст + Описание

In [102]:
cv_tensor = concat_tensors((cv_numb, cv_other, cv_desc))

for i in range(3):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])
    
    query_tensor = concat_tensors((query_numb, query_other, query_desc))[0]
    
    hits = util.semantic_search(query_tensor, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(3):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. 4', 1.0]

['Технический писатель. Королев. 40000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, удаленная работа. 4', 1.0]

['Оператор. Тверь. 20000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

['Веб-разработчик (HTML / CSS / JS / PHP / базы данных; фреймворки, дизайн, интерфейсы, CMS). Саратов. 100000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, удаленная работа. 4', 1.0]

['Региональный менеджер по продажам. Москва. 140000.0. От 3 до 6 лет. Полная занятость, полный день. 3', 1.0]

-------------------------------------------------------------------------------------------------------

#### multilingual-e5-large. Числовые + Текст

In [107]:
cv_tensor = concat_tensors((cv_other, cv_numb))

for i in range(3):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])
    
    query_tensor = concat_tensors((query_other, query_numb))
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=10)
    hits = hits[0] 
    
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(5):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([1, 1026])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. 4', 1.0]

['Технический писатель. Королев. 40000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, удаленная работа. 4', 1.0]

['Оператор. Тверь. 20000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

['Веб-разработчик (HTML / CSS / JS / PHP / базы данных; фреймворки, дизайн, интерфейсы, CMS). Саратов. 100000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, удаленная работа. 4', 1.0]

['Региональный менеджер по продажам. Москва. 140000.0. От 3 до 6 лет. Полная занятость, полный день. 3', 1.0]

---------------------------------------------------------------------------------

### multilingual-e5-large. Текст

In [244]:
cv_tensor = cv_other

print('ME5L | БЕЗ ОПИСАНИЯ (ТОЛЬКО ТЕКСТ)')
print()
for i in range(4):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])
    
    query_tensor = query_other
    
    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=11)
    hits = hits[0] 
    
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()
    print('--------------------------------------------------------------------------------------------------------------------')

ME5L | БЕЗ ОПИСАНИЯ (ТОЛЬКО ТЕКСТ)

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Начальник участка. Москва. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

['Инженер,начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9585562944412231]

--------------------------------------------------------------------------------------------------------------------
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий инженер. Владивосток. 60000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9497512578964233]

['Ведущий инженер. Владивосток. 95000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9497511386871338]

--------------------------------------------------------------------------------------------------------------------
Директор по персоналу (HRD). Владимир. 300000.0. Более 6 лет. Полная занятость, удаленная работа. 4

['Менеджер. Вл

#### multilingual-e5-large. Описание

In [151]:
cv_tensor = cv_desc

print('ME5L | ОПИСАНИЕ')
print()
for i in range(4):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])
    
    query_tensor = query_desc
    
    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=10)
    hits = hits[0] 
    
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

ME5L | ОПИСАНИЕ

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.8596729636192322]

['Менеджер проектов. Санкт-Петербург. 60000.0. Более 6 лет. Проектная работа, полная занятость, гибкий график, полный день. 4', 0.8572688102722168]

--------------------------------------------------------------------------------------------------------------------
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Аналитик. Москва. 55000.0. От 3 до 6 лет. Полная занятость, полный день. 3', 0.8703528642654419]

['Инженер-проектировщик. Омск. 40000.0. Более 6 лет. Полная занятость, полный день. 4', 0.868844747543335]

--------------------------------------------------------------------------------------------------------------------
Директор по персоналу (HRD). Владимир. 300000.0. Более 6 лет. Полная занято

#### multilingual-e5-large. (Числовые + Текст)

In [173]:
cv_tensor = cv_other_all

print('ME5L | БЕЗ ОПИСАНИЯ (ЧИСЛА+ТЕКСТ)')
print()
for i in range(4):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])
    
    query_tensor = query_other_all
    
    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=10)
    hits = hits[0] 
    
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

ME5L | БЕЗ ОПИСАНИЯ (ЧИСЛА+ТЕКСТ)

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Начальник участка. Москва. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9661480188369751]

['Начальник отдела(управления). Москва. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9559116363525391]

--------------------------------------------------------------------------------------------------------------------
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Руководитель проекта. Владивосток. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9521461725234985]

['ведущий инженер. Владивосток. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.946925938129425]

--------------------------------------------------------------------------------------------------------------------
Директор по персоналу (HRD). Владимир. 300000.0. Более 6 лет. Полная занятость, удаленная раб

#### multilingual-e5-large. (Числовые + Текст) + Описание

In [170]:
cv_tensor = concat_tensors((cv_other_all, cv_desc))

print('ME5L | ОПИСАНИЕ + ОСТАЛЬНЫЕ')
print()
for i in range(4):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])
    
    query_tensor = concat_tensors((query_other_all, query_desc))
    
    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[base_query, cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

ME5L | ОПИСАНИЕ + ОСТАЛЬНЫЕ

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9034316539764404]

['Начальник участка. Москва. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9002269506454468]

--------------------------------------------------------------------------------------------------------------------
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий инженер-проектировщик. Москва. 90000.0. От 3 до 6 лет. Полная занятость, полный день. 3', 0.8912509083747864]

['Ведущий инженер-проектировщик. Москва. 90000.0. Более 6 лет. Полная занятость, удаленная работа, полный день. 4', 0.8898914456367493]

--------------------------------------------------------------------------------------------------------------------
Директор по персоналу (HRD). Владимир. 300000.0. Более 

In [136]:
cols = ['title', 'location', 'salary', 'experience_group','job_type']
cv_my_ft = get_emb_fasttext(cv, cols=cols)
cv_tensor = cv_my_ft

for i in range(5):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_ft = get_emb_fasttext(query, cols=cols)

    query_tensor = query_other_all[0]
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_ft, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([300])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Руководитель отдела. Москва. 80000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9975993037223816]

['Руководитель проектов. Москва. 50000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9969857335090637]

--------------------------------------------------------------------------------------------------------------------
torch.Size([300])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий программист. Калининград. 100000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9986531138420105]

['Ведущий специалист. Ставрополь. 30000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9986056685447693]

--------------------------------------------------------------------------------------------------------------------
torch.Size([300])
Директор по персоналу (HRD). Владимир. 300000.0. Б

In [142]:
cols = ['title', 'location','job_type']

cv_my_ft = get_emb_fasttext(cv, cols=cols)

cv_tensor = cv_my_ft

for i in range(5):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_ft = get_emb_fasttext(query, cols=cols)

    query_tensor = query_other_all[0]
    print(query_tensor.shape)
    
    hits = util.semantic_search(query_ft, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(3):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

torch.Size([300])
Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Начальник участка. Москва. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 1.0]

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9853485822677612]

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9853485822677612]

--------------------------------------------------------------------------------------------------------------------
torch.Size([300])
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий программист 1C. Владивосток. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9849994778633118]

['Ведущий программист 1C. Владивосток. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9849994778633118]

['Ведущий специалист-руководитель. Санкт-Петербург. 80000.0. Более 6 лет

In [140]:
cols = ['title', 'location']

cv_my_ft = get_emb_fasttext(cv, cols=cols)

cv_tensor = cv_my_ft

for i in range(5):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_ft = get_emb_fasttext(query, cols=cols)

    query_tensor = query_other_all[0]
    
    hits = util.semantic_search(query_ft, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Начальник участка. Москва. 90000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9999998211860657]

['Руководитель проекта, начальник участка. Москва. 100000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9374547004699707]

--------------------------------------------------------------------------------------------------------------------
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий программист 1C. Владивосток. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9152354001998901]

['Ведущий программист 1C. Владивосток. 150000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9152354001998901]

--------------------------------------------------------------------------------------------------------------------
Директор по персоналу (HRD). Владимир. 300000.0. Более 6 лет. Полная занятость, удаленная работа. 4

['Дир

In [141]:
cols = ['title', 'location','experience_group']

cv_my_ft = get_emb_fasttext(cv, cols=cols)

cv_tensor = cv_my_ft

for i in range(5):
    base_query = vacancy.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = vacancy.iloc[[i]]
    
    query_ft = get_emb_fasttext(query, cols=cols)

    query_tensor = query_other_all[0]
    
    hits = util.semantic_search(query_ft, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

Начальник участка. Москва. 130000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Начальник отдела. Москва. 40000.0. От 1 года до 3 лет. Стажировка, волонтерство, частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. 2', 0.9979612827301025]

['Руководитель отдела. Москва. 80000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9975120425224304]

--------------------------------------------------------------------------------------------------------------------
Ведущий архитектор. Владивосток. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2

['Ведущий специалист. Ставрополь. 30000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.998836874961853]

['Ведущий программист. Калининград. 100000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9986987113952637]

-------------------------------------------------------------------------------------------------

In [229]:
def m5e_predict_cv(query):
    cv_tensor = cv_other_all
    base_query = query.drop('description',axis=1).apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    print('ME5L | БЕЗ ОПИСАНИЯ (ЧИСЛА+ТЕКСТ)')

    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_e5_emb(query[['title', 'location', 'job_type']])
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    query_desc = get_e5_emb(query[['description']])

    query_tensor = query_other_all

    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=10)
    hits = hits[0] 

    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')
    
def ft_predict(query):
    cv_tensor = cv_other_all_ft

    print('FastText | БЕЗ ОПИСАНИЯ')
    print()

    query_numb = df_to_tensor(query[['salary','exp_group_num']])
    query_other = get_emb_fasttext(query, cols=['title', 'location', 'job_type'])
    query_other_all = get_emb_fasttext(query, cols=['salary','exp_group_num','title', 'location', 'job_type'])
    query_desc = get_emb_fasttext(query, cols=['description'])

    query_tensor = query_other_all[0]
    print(query_tensor.shape)

    hits = util.semantic_search(query_tensor, cv_tensor, top_k=10)
    hits = hits[0] 
    cross_inp_res = [[cv_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(2):
        print(cross_inp_res[i])
        print()

    print('--------------------------------------------------------------------------------------------------------------------')

In [241]:
query = pd.DataFrame([{
    'title': 'Data Scientist', 
    'location': 'Новосибирск', 
    'salary': 777777,
    'experience_group': 'От 1 до 3 лет', 
    'job_type': 'Удаленно', 
    'description': '',
    'exp_group_num': 2
        }])
query

,title,location,salary,experience_group,job_type,description,exp_group_num
0,Data Scientist,Новосибирск,777777,От 1 до 3 лет,Удаленно,,2


In [242]:
m5e_predict(query)

ME5L | БЕЗ ОПИСАНИЯ (ЧИСЛА+ТЕКСТ)
Data Scientist. Новосибирск. 777777. От 1 до 3 лет. Удаленно. 2

['Data-scientist. Новосибирск. 30000.0. Более 6 лет. Стажировка, частичная занятость, полная занятость, гибкий график, полный день, сменный график, удаленная работа. 4', 0.9244948625564575]

['Аналитик. Новосибирск. 70000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9131844639778137]

--------------------------------------------------------------------------------------------------------------------


In [240]:
ft_predict(query)

FastText | БЕЗ ОПИСАНИЯ

torch.Size([300])
Data Scientist. SP. 777777. От 1 до 3 лет. Полный день. 2

['Data Scientist. Санкт-Петербург. 70000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.959152102470398]

['Data Scientist. Москва. 90000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9548285603523254]

--------------------------------------------------------------------------------------------------------------------


### Предсказание вакансий

In [247]:
vac_other_all = get_e5_emb(vacancy[['salary','exp_group_num','title', 'location', 'job_type']])

In [250]:
vac_list = vacancy.drop('description',axis=1).apply(lambda row: '. '.join(row.astype(str)), axis=1).values

In [255]:
cv_tensor = vac_other_all

print('ME5L | БЕЗ ОПИСАНИЯ (ЧИСЛА+ТЕКСТ)')
print()
for i in range(4):
    base_query = cv.drop('description',axis=1).iloc[[i]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]
    query = cv.iloc[[i]]
    
    query_other_all = get_e5_emb(query[['salary','exp_group_num','title', 'location', 'job_type']])
    
    query_tensor = query_other_all
    
    hits = util.semantic_search(query_tensor[0], cv_tensor, top_k=15)
    hits = hits[0] 
    cross_inp_res = [[base_query, vac_list[hit['corpus_id']], hit['score']] for hit in hits]
    cross_inp_res = [[vac_list[hit['corpus_id']], hit['score']] for hit in hits]
    print(base_query)
    print()
    for i in range(15):
        print(i,cross_inp_res[i])
        print()
    break
    print('--------------------------------------------------------------------------------------------------------------------')

ME5L | БЕЗ ОПИСАНИЯ (ЧИСЛА+ТЕКСТ)

Системный администратор. Советск. 29000.0. Более 6 лет. Частичная занятость, проектная работа, полная занятость, гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. 4

0 ['Системный администратор. Новосибирск. 39000.0. Нет опыта. Полная занятость, полный день. 1', 0.9195109605789185]

1 ['Системный администратор. Симферополь. 29000.0. Нет опыта. Полная занятость, полный день. 1', 0.9183518886566162]

2 ['Системный администратор. Дегтярск. 49000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9162078499794006]

3 ['Системный администратор. Санкт-Петербург. 25000.0. Более 6 лет. Частичная занятость, удаленная работа. 4', 0.9137195348739624]

4 ['Системный администратор. Новосибирск. 65000.0. От 1 года до 3 лет. Полная занятость, полный день. 2', 0.9132810235023499]

5 ['Системный администратор. Новосибирск. 50000.0. Более 6 лет. Полная занятость, полный день. 4', 0.9126412272453308]

6 ['Системный администратор (уда

In [257]:
vacancy[(vacancy.title=='Системный администратор')&(vacancy.location=='Калининград')]

,title,location,salary,experience_group,job_type,description,exp_group_num
60211,Системный администратор,Калининград,115000.0,Более 6 лет,"Полная занятость, полный день",Обязанности: обеспечение бесперебойно...,4
375793,Системный администратор,Калининград,75000.0,От 1 года до 3 лет,"Полная занятость, полный день","«SPAR» ищет в свою команду коллегу, професси...",2
413088,Системный администратор,Калининград,115000.0,Более 6 лет,"Полная занятость, полный день",Обязанности: обеспечение бесперебойно...,4
544676,Системный администратор,Калининград,90000.0,От 3 до 6 лет,"Полная занятость, полный день",Обязанности: разработка/доработка фун...,3
